In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

simulation_backend = "qasm_simulator"

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

100000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30523/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐       ┌────────────────┐┌────────────┐   ┌─────────────────┐»
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ └─────────────────┘»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                           │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────────────────────────■─────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                      »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘                                    »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_6: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌──────────────┐                                                     »
«q_0: ─┤ Rz(-0.23807) ├─────────────────────────────────────────────────────»
«     ┌┴──────────────┤┌──────────────┐                                     »
«q_1: ┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├─────────────────────────────────────»
«     └───────────────┘├─────────────┬┘  ┌────────────┐                     »
«q_2: ────────■────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├─────────────────────»
«             │        └─────────────┘ ┌─┴────────────┴─┐  ┌──────────────┐ »
«q_3: ────────■───────────────■────────┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├─»
«                             │        └────────────────┘  ├──────────────┤ »
«q_4: ────────────────────────■────────────────■───────────┤ Ry(0.025251) ├─»
«                                              │         ┌─┴──────────────┴┐»
«q_5: ─────────────────────────────────────────■─────────┤ Ry(-1.0324e-05) ├»
«                                                        └─────────────────┘»
«q_6: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«                      
«q_2: ─────────────────
«                      
«q_3: ─────────────────
«       ┌───────────┐  
«q_4: ──┤ Rz(1.011) ├──
«     ┌─┴───────────┴─┐
«q_5: ┤ Rz(-0.046623) ├
«     └───────────────┘
«q_6: ─────────────────
«

In [16]:
#checking circuit

In [17]:
phase_y_front = [2.113295958493177e-07,0.0054976714345856634, 0.020630778702533964, 3.40008274438278e-05, 0.02524649597343495, -4.81317308598186e-05]
phase_z_front = [1.3165147216569606, -0.07815775859934079, 0.006091440853078919, 1.528289688855932, -0.009893684371252858, 1.3655876587592497]
phase_y_end = [-7.269945363221252e-06, 0.0055856209568994186, 0.020669506733621706, 2.6996665334365784e-05, 0.02525084679082589, -1.0323843699871446e-05]
phase_z_end = [-0.23806644958543488, -0.21685946937956296, 1.2696203088392648, 0.02265810653600898, 1.010960593326885, -0.04662312860318999]
x_front = [True, False, False, True, False, False]

In [18]:
qubit_id = 4
circuit = QuantumCircuit(7)
apply_checking_circuit(circuit, [qubit_id], [6], side = 'front', phase_y = phase_y_front[qubit_id], phase_z = phase_z_front[qubit_id], x = x_front[qubit_id])
circuit += orign_circuit
apply_checking_circuit(circuit, [qubit_id], [6], side = 'end', phase_y = phase_y_end[qubit_id], phase_z = phase_z_end[qubit_id])

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30523/340002502.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [19]:
circuit.draw()

┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├───────────────────────────────■────────────────────────»
            └───┘                                                        »
«     ┌─────────────────┐ ┌──────────────┐                                »
«q_0: ┤ Ry(-7.2699e-06) ├─┤ Rz(-0.23807) ├────────────────────────────────»
«     └─────────────────┘┌┴──────────────┤ ┌──────────────┐               »
«q_1: ─────────■─────────┤ Ry(0.0055856) ├─┤ Rz(-0.21686) ├───────────────»
«              │         └───────────────┘ ├─────────────┬┘ ┌────────────┐»
«q_2: ─────────■─────────────────■─────────┤ Ry(0.02067) ├──┤ Rz(1.2696) ├»
«                                │         └─────────────┘  └────────────┘»
«q_3: ───────────────────────────■────────────────────────────────■───────»
«      ┌───────────────┐  ┌──────────────┐┌────────────────┐      │       »
«q_4: ─┤ Ry(-0.025246) ├──┤ Ry(0.025246) ├┤ Rz(-0.0098937) ├──────■───────»
«      └───────────────┘  └──────────────┘└────────────────┘              »
«q_5: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«q_6: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«                                                                         »
«q_0: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«q_1: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«q_2: ────────────────────────────────────────────────────────────────────»
«     ┌────────────────┐  ┌──────────────┐                                »
«q_3: ┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├────────────────────────────────»
«     └────────────────┘  ├──────────────┤   ┌───────────┐  ┌────────────┐»
«q_4: ────────■───────────┤ Ry(0.025251) ├───┤ Rz(1.011) ├──┤ Rz(-1.011) ├»
«             │         ┌─┴──────────────┴┐┌─┴───────────┴─┐└────────────┘»
«q_5: ────────■─────────┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├──────────────»
«                       └─────────────────┘└───────────────┘              »
«q_6: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«                                                      
«q_0: ─────────────────────────────────────────────────
«                                                      
«q_1: ─────────────────────────────────────────────────
«                                                      
«q_2: ─────────────────────────────────────────────────
«                                                      
«q_3: ─────────────────────────────────────────────────
«     ┌───────────────┐   ┌──────────────┐┌───────────┐
«q_4: ┤ Ry(-0.025251) ├─■─┤ Ry(0.025251) ├┤ Rz(1.011) ├
«     └───────────────┘ │ └──────────────┘└────

In [20]:
pauli_list = read_from_file('BeH2_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [21]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 6,num_qargs = 7)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YXYYXY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
                     
YXYZXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_3: ────────────────
      ┌────────────┐ 
q_4: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_5: ────────────────
                     
q_6: ────────────────
                     
ZXZYXY
                     
q_0: ────────────────
      ┌────────────┐ 
q_1: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_2: ────────────────
     ┌──────────────┐
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
           

In [22]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30523/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [23]:
circuit_list[-2].draw()

┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├───────────────────────────────■────────────────────────»
            └───┘                                                        »
«     ┌─────────────────┐ ┌──────────────┐ ┌──────────────┐                 »
«q_0: ┤ Ry(-7.2699e-06) ├─┤ Rz(-0.23807) ├─┤ U(π/2,0,π/2) ├─────────────────»
«     └─────────────────┘┌┴──────────────┤ ├──────────────┤ ┌──────────────┐»
«q_1: ─────────■─────────┤ Ry(0.0055856) ├─┤ Rz(-0.21686) ├─┤ U(π/2,0,π/2) ├»
«              │         └───────────────┘ ├─────────────┬┘ └┬────────────┬┘»
«q_2: ─────────■─────────────────■─────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├─»
«                                │         └─────────────┘   └────────────┘ »
«q_3: ───────────────────────────■─────────────────────────────────■────────»
«      ┌───────────────┐  ┌──────────────┐┌────────────────┐       │        »
«q_4: ─┤ Ry(-0.025246) ├──┤ Ry(0.025246) ├┤ Rz(-0.0098937) ├───────■────────»
«      └───────────────┘  └──────────────┘└────────────────┘                »
«q_5: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«q_6: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                                                                         »
«q_0: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«q_1: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«q_2: ────────────────────────────────────────────────────────────────────»
«     ┌────────────────┐  ┌──────────────┐  ┌──────────────┐              »
«q_3: ┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├──┤ U(π/2,0,π/2) ├──────────────»
«     └────────────────┘  ├──────────────┤  └┬───────────┬─┘┌────────────┐»
«q_4: ────────■───────────┤ Ry(0.025251) ├───┤ Rz(1.011) ├──┤ Rz(-1.011) ├»
«             │         ┌─┴──────────────┴┐┌─┴───────────┴─┐└────────────┘»
«q_5: ────────■─────────┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├──────────────»
«                       └─────────────────┘└───────────────┘              »
«q_6: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«                                                                      
«q_0: ─────────────────────────────────────────────────────────────────
«                                                                      
«q_1: ─────────────────────────────────────────────────────────────────
«                                                                      
«q_2: ─────────────────────────────────────────────────────────────────
«                                                                      
«q_3: ─────────────────────────────────────────────────────────────────
«

In [24]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [25]:
cuts = [(circuit.qubits[4], 5),(circuit.qubits[4], 11)]

In [26]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

...

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

            ┌───┐       ┌────────────────┐┌────────────┐                 »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │        »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■────────»
      └┬──────────────┤ ├───────────────┤                                »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                 »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘┌───────────────┐»
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────■───────┤ Rz(0.0098937) ├»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │       └───────────────┘»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                        »
q_6: ───────┤ H ├────────

In [27]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(7, 'q'), 0): ((0, Qubit(QuantumRegister(6, 'q'), 5)),), Qubit(QuantumRegister(7, 'q'), 1): ((0, Qubit(QuantumRegister(6, 'q'), 2)),), Qubit(QuantumRegister(7, 'q'), 2): ((0, Qubit(QuantumRegister(6, 'q'), 3)),), Qubit(QuantumRegister(7, 'q'), 3): ((0, Qubit(QuantumRegister(6, 'q'), 0)),), Qubit(QuantumRegister(7, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 2)), (0, Qubit(QuantumRegister(6, 'q'), 1)), (1, Qubit(QuantumRegister(3, 'q'), 1))), Qubit(QuantumRegister(7, 'q'), 5): ((0, Qubit(QuantumRegister(6, 'q'), 4)),), Qubit(QuantumRegister(7, 'q'), 6): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}


In [28]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [29]:
total_variants

24

In [30]:
shots = 10000 * total_variants

In [31]:
fragments_list[0][0].draw()

┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_1: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
      ┌┴──────────────┤ ├───────────────┬┘                                    »
q_2: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_3: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_4: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └────────────────┘└────────────┘   └─────────────────┘»
«                                                      ┌────────────────┐»
«q_0: ────────────────────────■───────────────■────────┤ Ry(2.6997e-05) ├»
«                             │               │        └────────────────┘»
«q_1: ────────────────────────┼───────────────■──────────────────────────»
«     ┌───────────────┐       │        ┌──────────────┐  ┌────────────┐  »
«q_2: ┤ Ry(0.0055856) ├───────┼────────┤ Rz(-0.21686) ├──┤ U(π/2,0,π) ├──»
«     └───────────────┘       │        ├─────────────┬┘  ├────────────┤  »
«q_3: ────────────────────────■────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├──»
«                                      └─────────────┘   └────────────┘  »
«q_4: ───────────────────────────────────────────────────────────────────»
«      ┌──────────────┐┌──────────────┐                                  »
«q_5: ─┤ Rz(-0.23807) ├┤ U(π/2,0,π/2) ├──────────────────────────────────»
«      └──────────────┘└──────────────┘                                  »
«     ┌──────────────┐  ┌──────────────┐                                  
«q_0: ┤ Rz(0.022658) ├──┤ U(π/2,0,π/2) ├──────────────────────────────────
«     └──────────────┘  ├──────────────┤   ┌───────────┐                  
«q_1: ───────■──────────┤ Ry(0.025251) ├───┤ Rz(1.011) ├──────────────────
«            │          └──────────────┘   └───────────┘                  
«q_2: ───────┼────────────────────────────────────────────────────────────
«            │          ┌──────────────┐                                  
«q_3: ───────┼──────────┤ U(π/2,0,π/2) ├──────────────────────────────────
«            │        ┌─┴──────────────┴┐┌───────────────┐┌──────────────┐
«q_4: ───────■────────┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├┤ U(π/2,0,π/2) ├
«                     └─────────────────┘└───────────────┘└──────────────┘
«q_5: ────────────────────────────────────────────────────────────────────
«

In [32]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_1: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
      ┌┴──────────────┤ ├───────────────┬┘                                    »
q_2: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_3: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_4: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └───

            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_1: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
      ┌┴──────────────┤ ├───────────────┬┘                                    »
q_2: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_3: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_4: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └───────────────

          ┌───┐                                                  ┌───┐      »
q_0: ─────┤ H ├─────────────────────────■─────────■──────────────┤ H ├──────»
      ┌───┴───┴────┐ ┌───────────────┐  │         │         ┌────┴───┴─────┐»
q_1: ─┤ Rz(-1.011) ├─┤ Ry(-0.025251) ├──┼─────────■─────────┤ Ry(0.025251) ├»
     ┌┴────────────┴┐├───────────────┴┐ │ ┌───────────────┐┌┴──────────────┤»
q_2: ┤ Ry(0.025246) ├┤ Rz(-0.0098937) ├─■─┤ Rz(0.0098937) ├┤ Ry(-0.025246) ├»
     └──────────────┘└────────────────┘   └───────────────┘└───────────────┘»
«                                
«q_0: ───────────────────────────
«     ┌───────────┐┌────────────┐
«q_1: ┤ Rz(1.011) ├┤ U(π/2,0,π) ├
«     └───────────┘└────────────┘
«q_2: ───────────────────────────
«                                

fragment 0:
            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└

            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_1: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
      ┌┴──────────────┤ ├───────────────┬┘                                    »
q_2: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_3: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_4: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └───────────────

In [33]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [34]:
wire_path_map

{Qubit(QuantumRegister(7, 'q'), 0): ((0, Qubit(QuantumRegister(6, 'q'), 5)),),
 Qubit(QuantumRegister(7, 'q'), 1): ((0, Qubit(QuantumRegister(6, 'q'), 2)),),
 Qubit(QuantumRegister(7, 'q'), 2): ((0, Qubit(QuantumRegister(6, 'q'), 3)),),
 Qubit(QuantumRegister(7, 'q'), 3): ((0, Qubit(QuantumRegister(6, 'q'), 0)),),
 Qubit(QuantumRegister(7, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 2)),
  (0, Qubit(QuantumRegister(6, 'q'), 1)),
  (1, Qubit(QuantumRegister(3, 'q'), 1))),
 Qubit(QuantumRegister(7, 'q'), 5): ((0, Qubit(QuantumRegister(6, 'q'), 4)),),
 Qubit(QuantumRegister(7, 'q'), 6): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}

In [35]:
num_fragments = len(fragments)
num_fragments

2

In [36]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

34

In [37]:
fragments[0].draw()

┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_1: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
      ┌┴──────────────┤ ├───────────────┬┘                                    »
q_2: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_3: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_4: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └────────────────┘└────────────┘   └─────────────────┘»
«                                         ┌────────────────┐┌──────────────┐»
«q_0: ──────────────────■────────■────────┤ Ry(2.6997e-05) ├┤ Rz(0.022658) ├»
«                       │        │        └────────────────┘└──────────────┘»
«q_1: ──────────────────┼────────■─────────────────────────────────■────────»
«     ┌───────────────┐ │ ┌──────────────┐ ┌──────────────┐        │        »
«q_2: ┤ Ry(0.0055856) ├─┼─┤ Rz(-0.21686) ├─┤ U(π/2,0,π/2) ├────────┼────────»
«     └───────────────┘ │ ├─────────────┬┘ └┬────────────┬┘        │        »
«q_3: ──────────────────■─┤ Ry(0.02067) ├───┤ Rz(1.2696) ├─────────┼────────»
«                         └─────────────┘   └────────────┘         │        »
«q_4: ─────────────────────────────────────────────────────────────■────────»
«      ┌──────────────┐                                                     »
«q_5: ─┤ Rz(-0.23807) ├─────────────────────────────────────────────────────»
«      └──────────────┘                                                     »
«                                                         
«q_0: ────────────────────────────────────────────────────
«       ┌──────────────┐   ┌───────────┐                  
«q_1: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├──────────────────
«       └──────────────┘   └───────────┘                  
«q_2: ────────────────────────────────────────────────────
«       ┌──────────────┐                                  
«q_3: ──┤ U(π/2,0,π/2) ├──────────────────────────────────
«     ┌─┴──────────────┴┐┌───────────────┐┌──────────────┐
«q_4: ┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├┤ U(π/2,0,π/2) ├
«     └─────────────────┘└───────────────┘└──────────────┘
«q_5: ────────────────────────────────────────────────────
«

In [38]:
fragments[1].draw()

┌───┐                                                  ┌───┐      »
q_0: ─────┤ H ├─────────────────────────■─────────■──────────────┤ H ├──────»
      ┌───┴───┴────┐ ┌───────────────┐  │         │         ┌────┴───┴─────┐»
q_1: ─┤ Rz(-1.011) ├─┤ Ry(-0.025251) ├──┼─────────■─────────┤ Ry(0.025251) ├»
     ┌┴────────────┴┐├───────────────┴┐ │ ┌───────────────┐┌┴──────────────┤»
q_2: ┤ Ry(0.025246) ├┤ Rz(-0.0098937) ├─■─┤ Rz(0.0098937) ├┤ Ry(-0.025246) ├»
     └──────────────┘└────────────────┘   └───────────────┘└───────────────┘»
«                                  
«q_0: ─────────────────────────────
«     ┌───────────┐┌──────────────┐
«q_1: ┤ Rz(1.011) ├┤ U(π/2,0,π/2) ├
«     └───────────┘└──────────────┘
«q_2: ─────────────────────────────
«

In [39]:
hardware_index = 0

In [40]:
print(shots)

240000


In [41]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [42]:
#contains 34 basis
len(tomography_circuits)

34

In [43]:
#contains two fragments
len(tomography_circuits[0])

2

In [44]:
len(tomography_circuits[0][1])

1

In [45]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [46]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [47]:
#[12,13,14,11,8,9]
#[11,8,13,14,9,12]

In [49]:
multi_exp_circuits[hardware_index][-200].draw()

░        ┌───┐       ┌────────────────┐»
  q_0: ────────────────────░────────┤ X ├───────┤ Ry(3.4001e-05) ├»
       ┌─────────────────┐ ░   ┌────┴───┴─────┐ ├────────────────┤»
  q_1: ┤ U3(-1.9106,π,0) ├─░───┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├»
       └─────────────────┘ ░  ┌┴──────────────┤ ├───────────────┬┘»
  q_2: ────────────────────░──┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├─»
                           ░  └┬──────────────┤ ├───────────────┤ »
  q_3: ────────────────────░───┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├─»
                           ░ ┌─┴──────────────┴┐└─┬────────────┬┘ »
  q_4: ────────────────────░─┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──»
                           ░ └──────┬───┬──────┘┌─┴────────────┴─┐»
  q_5: ────────────────────░────────┤ X ├───────┤ Ry(2.1133e-07) ├»
                           ░        └───┘       └────────────────┘»
c34_0: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_1: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_2: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_3: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_4: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_5: ═══════════════════════════════════════════════════════════»
                                                                  »
«       ┌────────────┐                                          »
«  q_0: ┤ Rz(1.5283) ├────────────────────────────────────────■─»
«       └────────────┘                                        │ »
«  q_1: ──────────────────────────────────────────────────────┼─»
«                                           ┌───────────────┐ │ »
«  q_2: ───────────────■──────────■─────────┤ Ry(0.0055856) ├─┼─»
«                      │          │         └───────────────┘ │ »
«  q_3: ───────────────┼──────────■───────────────────────────■─»
«                      │                                        »
«  q_4: ───────────────┼────────────────────────────────────────»
«       ┌────────────┐ │ ┌─────────────────┐ ┌──────────────┐   »
«  q_5: ┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├─┤ Rz(-0.23807) ├───»
«       └────────────┘   └─────────────────┘ └──────────────┘   »
«c34_0: ════════════════════════════════════════════════════════»
«                                                               »
«c34_1: ════════════════════════════════════════════════════════»
«                                                               »
«c34_2: ════════════════════════════════════════════════════════»
«                                                               »
«c34_3: ════════════════════════════════════════════════════════»
«                                                               »
«c34_4: ════════════════════════════════════════════════════════»
«                                                               »
«c34_5: ════════════════════════════════════════════════════════»
«                                                               »
«                       ┌────────────────┐┌──────────────┐  ┌──────────────┐ »
«  q_0: ───────■────────┤ Ry(2.6997e-05) ├┤ Rz(0.022658) ├──┤ U(π/2,0,π/2) ├─»
«              │        └────────────────┘└──────────────┘  ├──────────────┤ »
«  q_1: ───────■─────────────────────────────────■──────────┤ Ry(0.025251) ├─»
«       ┌──────────────┐  ┌────────────┐         │          └──────────────┘ »
«  q_2: ┤ Rz(-0.21686) ├──┤ U(π/2,0,π) ├─────────┼───────────────────────────»
«       ├─────────────┬┘  ├────────────┤         │           ┌────────────┐  »
«  q_3: ┤ Ry(0.02067) ├───┤ Rz(1.2696) ├─────────┼───────────┤ U(π/2,0,π) ├──»
«       └─────────────┘   └────────────┘     

In [50]:
len(multi_exp_circuits[0])

408

In [50]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [11,8,13,14,9,12],
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

jobset_id:  f54db6b13cce49bb9d0c85731e98af5a-1670513364197917
job_id:  6416a299-fd67-4ea5-851e-f9f31bc9f0d4
Job Status: job has successfully run


In [51]:
frag_datas[1]

[Result(backend_name='qasm_simulator', backend_version='0.11.1', qobj_id='49d1ff49-5d27-4a89-99b1-11587a7140b0', job_id='6416a299-fd67-4ea5-851e-f9f31bc9f0d4', success=True, results=[ExperimentResult(shots=10000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x4': 1, '0x1': 2, '0x7': 1, '0x3': 1, '0x5': 1, '0x0': 5091, '0x2': 4903}), header=QobjExperimentHeader(clbit_labels=[['c1', 0], ['c1', 1], ['c1', 2]], creg_sizes=[['c1', 3]], global_phase=0.01062696737806158, memory_slots=3, metadata={}, n_qubits=3, name="(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z'))", qreg_sizes=[['q', 3]], qubit_labels=[['q', 0], ['q', 1], ['q', 2]]), status=DONE, seed_simulator=3175012304, metadata={'parallel_state_update': 8, 'parallel_shots': 1, 'sample_measure_time': 0.003424779, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': False, 'device': 'CPU', 'active_input_qubits': [0, 1, 2], 'measure_sampling': True, 'num_clbits': 3, 'input_qubit_map': [[2, 2], [1, 1], [0, 0]], 'n

In [87]:
# from qiskit.providers.ibmq.managed import IBMQJobManager
# job_manager = IBMQJobManager()

In [88]:
# job_set = job_manager.retrieve_job_set(job_set_id='973c2fdceaf547389a770d19aad463d8-1669966038684177', provider=provider)

In [89]:
# managed_results = job_set.results()

In [90]:
# frag_datas1_copy = frag_datas[1].copy()

In [52]:
frag_datas[0] = [frag_datas[0][0].combine_results()]

In [53]:
# frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
#                              backend = simulation_backend, shots = shots // total_variants,
#                              optimization_level = 3, monitor_jobs = True, noise_model = None)

In [54]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z'))"

In [55]:
num_tomo_circuits

34

In [56]:
test = [[], []]

In [57]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [58]:
frag_datas[1][0].get_counts()

[{'100': 1, '001': 2, '111': 1, '011': 1, '101': 1, '000': 5091, '010': 4903},
 {'011': 3, '001': 1, '100': 2647, '110': 2461, '010': 2416, '000': 2472},
 {'111': 4, '010': 2402, '000': 2638, '110': 2371, '100': 2585},
 {'111': 1, '001': 3302, '010': 1652, '011': 3335, '000': 1710},
 {'101': 1607,
  '100': 854,
  '110': 891,
  '001': 1595,
  '000': 851,
  '010': 800,
  '111': 1744,
  '011': 1658},
 {'110': 885,
  '001': 1630,
  '100': 853,
  '111': 1624,
  '011': 1581,
  '000': 861,
  '010': 843,
  '101': 1723},
 {'100': 1, '001': 3325, '111': 1, '011': 3531, '000': 1610, '010': 1532},
 {'110': 819,
  '100': 897,
  '001': 1627,
  '010': 781,
  '000': 770,
  '101': 1678,
  '011': 1684,
  '111': 1744},
 {'110': 762,
  '100': 867,
  '001': 1673,
  '011': 1673,
  '111': 1773,
  '000': 765,
  '010': 857,
  '101': 1630},
 {'100': 1, '001': 3199, '101': 3, '000': 1697, '010': 1642, '011': 3458},
 {'101': 1724,
  '010': 820,
  '000': 837,
  '110': 839,
  '100': 852,
  '001': 1492,
  '111': 176

In [59]:
frag_datas[0][0].get_counts()

[{'000000': 305,
  '000001': 238,
  '010000': 250,
  '010001': 257,
  '010010': 1,
  '010011': 8,
  '010100': 462,
  '010101': 416,
  '010110': 13,
  '010111': 13,
  '011000': 262,
  '011001': 247,
  '011010': 5,
  '011011': 5,
  '011100': 438,
  '011101': 381,
  '011110': 10,
  '011111': 10,
  '000010': 12,
  '100000': 279,
  '100001': 197,
  '100010': 5,
  '100011': 2,
  '100100': 310,
  '100101': 303,
  '100110': 9,
  '100111': 5,
  '101000': 244,
  '101001': 213,
  '101010': 5,
  '101011': 6,
  '101100': 325,
  '101101': 271,
  '101110': 7,
  '101111': 4,
  '000011': 6,
  '110000': 218,
  '110001': 207,
  '110011': 3,
  '110100': 292,
  '110101': 248,
  '110110': 5,
  '110111': 7,
  '111000': 206,
  '111001': 212,
  '111010': 4,
  '111011': 1,
  '111100': 282,
  '111101': 268,
  '111110': 3,
  '111111': 10,
  '000100': 532,
  '000101': 451,
  '000110': 10,
  '000111': 9,
  '001000': 252,
  '001001': 274,
  '001010': 5,
  '001011': 11,
  '001100': 525,
  '001101': 417,
  '001110': 1

In [60]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [1] [1]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'X', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z', 'Z', 'Z')) ('S0',) ('X',) [1] [1]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Y', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z', 'Z', 'Z')) ('S0',) ('Y',) [1] [1]
(('S0', 'S1', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S1', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S1', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [1] [1]
(('S0', 'S1', 'S0', 'S0', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z', 'Z', 'Z'

In [61]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [62]:
len(organized_data)

34

In [63]:
organized_data[2]

[{'00000': {(('S0',), ('Zp',)): 98,
   (('S0',), ('Zm',)): 1,
   (('S0',), ('Xp',)): 44,
   (('S0',), ('Xm',)): 34,
   (('S0',), ('Yp',)): 48,
   (('S0',), ('Ym',)): 73,
   (('S1',), ('Zp',)): 41,
   (('S1',), ('Zm',)): 53,
   (('S1',), ('Xp',)): 27,
   (('S1',), ('Xm',)): 62,
   (('S1',), ('Yp',)): 11,
   (('S1',), ('Ym',)): 68,
   (('S2',), ('Zp',)): 36,
   (('S2',), ('Zm',)): 48,
   (('S2',), ('Xp',)): 19,
   (('S2',), ('Xm',)): 63,
   (('S2',), ('Yp',)): 74,
   (('S2',), ('Ym',)): 14,
   (('S3',), ('Zp',)): 32,
   (('S3',), ('Zm',)): 44,
   (('S3',), ('Xp',)): 87,
   (('S3',), ('Xm',)): 7,
   (('S3',), ('Yp',)): 46,
   (('S3',), ('Ym',)): 46},
  '00001': {(('S0',), ('Zp',)): 94,
   (('S0',), ('Zm',)): 3,
   (('S0',), ('Xp',)): 51,
   (('S0',), ('Xm',)): 38,
   (('S0',), ('Yp',)): 52,
   (('S0',), ('Ym',)): 50,
   (('S1',), ('Zp',)): 31,
   (('S1',), ('Zm',)): 55,
   (('S1',), ('Xp',)): 37,
   (('S1',), ('Xm',)): 54,
   (('S1',), ('Yp',)): 12,
   (('S1',), ('Ym',)): 77,
   (('S2',),

In [64]:
organized_data[-1][-1]

{'11': {(('S0',), ('Zm',)): 1,
  (('S0',), ('Xp',)): 3,
  (('S0',), ('Yp',)): 1,
  (('S1',), ('Zp',)): 3429,
  (('S1',), ('Xm',)): 1677,
  (('S1',), ('Xp',)): 1663,
  (('S1',), ('Yp',)): 1579,
  (('S1',), ('Ym',)): 1752,
  (('S2',), ('Zp',)): 3445,
  (('S2',), ('Xp',)): 1708,
  (('S2',), ('Xm',)): 1743,
  (('S2',), ('Yp',)): 1757,
  (('S2',), ('Ym',)): 1742,
  (('S3',), ('Zp',)): 3414,
  (('S3',), ('Zm',)): 2,
  (('S3',), ('Xm',)): 1819,
  (('S3',), ('Xp',)): 1620,
  (('S3',), ('Ym',)): 1646,
  (('S3',), ('Yp',)): 1608,
  (('S0',), ('Zp',)): 0,
  (('S0',), ('Xm',)): 0,
  (('S0',), ('Ym',)): 0,
  (('S1',), ('Zm',)): 0,
  (('S2',), ('Zm',)): 0},
 '01': {(('S0',), ('Zm',)): 1,
  (('S0',), ('Zp',)): 1,
  (('S0',), ('Xm',)): 4,
  (('S0',), ('Ym',)): 1,
  (('S1',), ('Zp',)): 3202,
  (('S1',), ('Xm',)): 1663,
  (('S1',), ('Xp',)): 1602,
  (('S1',), ('Yp',)): 1672,
  (('S1',), ('Ym',)): 1573,
  (('S2',), ('Zm',)): 2,
  (('S2',), ('Zp',)): 3358,
  (('S2',), ('Xm',)): 1723,
  (('S2',), ('Xp',)):

In [65]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'00000': {(('S0',), ('Zp',)): 305, (('S0',), ('Zm',)): 12, (('S0',), ('Xp',)): 125, (('S0',), ('Xm',)): 145, (('S0',), ('Yp',)): 359, (('S0',), ('Ym',)): 332, (('S1',), ('Zp',)): 100, (('S1',), ('Zm',)): 138, (('S1',), ('Xp',)): 174, (('S1',), ('Xm',)): 501, (('S1',), ('Yp',)): 107, (('S1',), ('Ym',)): 536, (('S2',), ('Zp',)): 80, (('S2',), ('Zm',)): 180, (('S2',), ('Xp',)): 211, (('S2',), ('Xm',)): 475, (('S2',), ('Yp',)): 597, (('S2',), ('Ym',)): 65, (('S3',), ('Zp',)): 86, (('S3',), ('Zm',)): 176, (('S3',), ('Xp',)): 591, (('S3',), ('Xm',)): 46, (('S3',), ('Yp',)): 321, (('S3',), ('Ym',)): 382}, '00001': {(('S0',), ('Zp',)): 238, (('S0',), ('Zm',)): 6, (('S0',), ('Xp',)): 124, (('S0',), ('Xm',)): 136, (('S0',), ('Yp',)): 347, (('S0',), ('Ym',)): 320, (('S1',), ('Zp',)): 75, (('S1',), ('Zm',)): 155, (('S1',), ('Xp',)): 165, (('S1',), ('Xm',)): 522, (('S1',), ('Yp',)): 82, (('S1',), ('Ym',)): 562, (('S2',), ('Zp',)): 85, (('S2',), ('Zm',)): 162, (('S2',), ('Xp',)): 190, (('S2',), ('X

{'10': {(('S0',), ('Zm',)): 1, (('S0',), ('Zp',)): 4969, (('S0',), ('Xm',)): 2599, (('S0',), ('Xp',)): 2360, (('S0',), ('Ym',)): 2487, (('S0',), ('Yp',)): 2526, (('S1',), ('Zm',)): 2, (('S1',), ('Zp',)): 1604, (('S1',), ('Xm',)): 876, (('S1',), ('Xp',)): 793, (('S1',), ('Ym',)): 825, (('S1',), ('Yp',)): 794, (('S2',), ('Zm',)): 1, (('S2',), ('Zp',)): 1536, (('S2',), ('Xp',)): 760, (('S2',), ('Xm',)): 843, (('S2',), ('Ym',)): 782, (('S2',), ('Yp',)): 826, (('S3',), ('Zp',)): 1670, (('S3',), ('Xp',)): 868, (('S3',), ('Xm',)): 831, (('S3',), ('Ym',)): 852, (('S3',), ('Yp',)): 883, (('S3',), ('Zm',)): 0}, '11': {(('S0',), ('Zp',)): 1, (('S0',), ('Ym',)): 3, (('S1',), ('Zm',)): 1, (('S1',), ('Zp',)): 3359, (('S1',), ('Xp',)): 1625, (('S1',), ('Xm',)): 1836, (('S1',), ('Yp',)): 1596, (('S1',), ('Ym',)): 1665, (('S2',), ('Zp',)): 3461, (('S2',), ('Xp',)): 1686, (('S2',), ('Xm',)): 1773, (('S2',), ('Yp',)): 1667, (('S2',), ('Ym',)): 1766, (('S3',), ('Zp',)): 3348, (('S3',), ('Xp',)): 1633, (('

{'10': {(('S0',), ('Zm',)): 1, (('S0',), ('Zp',)): 4962, (('S0',), ('Xp',)): 2383, (('S0',), ('Xm',)): 2544, (('S0',), ('Ym',)): 2446, (('S0',), ('Yp',)): 2505, (('S1',), ('Zm',)): 1, (('S1',), ('Zp',)): 1667, (('S1',), ('Xm',)): 879, (('S1',), ('Xp',)): 827, (('S1',), ('Yp',)): 825, (('S1',), ('Ym',)): 807, (('S2',), ('Zp',)): 1563, (('S2',), ('Xm',)): 759, (('S2',), ('Xp',)): 782, (('S2',), ('Ym',)): 717, (('S2',), ('Yp',)): 878, (('S3',), ('Zp',)): 1600, (('S3',), ('Xp',)): 817, (('S3',), ('Xm',)): 785, (('S3',), ('Yp',)): 800, (('S3',), ('Ym',)): 848, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '01': {(('S0',), ('Zp',)): 1, (('S0',), ('Xm',)): 2, (('S0',), ('Yp',)): 3, (('S1',), ('Zp',)): 3222, (('S1',), ('Xp',)): 1533, (('S1',), ('Xm',)): 1655, (('S1',), ('Yp',)): 1667, (('S1',), ('Ym',)): 1641, (('S2',), ('Zp',)): 3384, (('S2',), ('Zm',)): 1, (('S2',), ('Xp',)): 1648, (('S2',), ('Xm',)): 1711, (('S2',), ('Yp',)): 1694, (('S2',), ('Ym',)): 1661, (('S3',), ('Zp',)): 3322, (('S3'

{'00000': {(('S0',), ('Zp',)): 257, (('S0',), ('Zm',)): 5, (('S0',), ('Xp',)): 134, (('S0',), ('Xm',)): 146, (('S0',), ('Yp',)): 376, (('S0',), ('Ym',)): 309, (('S1',), ('Zp',)): 100, (('S1',), ('Zm',)): 182, (('S1',), ('Xp',)): 193, (('S1',), ('Xm',)): 499, (('S1',), ('Yp',)): 93, (('S1',), ('Ym',)): 593, (('S2',), ('Zp',)): 92, (('S2',), ('Zm',)): 214, (('S2',), ('Xp',)): 222, (('S2',), ('Xm',)): 440, (('S2',), ('Yp',)): 590, (('S2',), ('Ym',)): 66, (('S3',), ('Zp',)): 87, (('S3',), ('Zm',)): 195, (('S3',), ('Xp',)): 562, (('S3',), ('Xm',)): 39, (('S3',), ('Yp',)): 317, (('S3',), ('Ym',)): 338}, '00001': {(('S0',), ('Zp',)): 280, (('S0',), ('Zm',)): 6, (('S0',), ('Xp',)): 130, (('S0',), ('Xm',)): 124, (('S0',), ('Yp',)): 337, (('S0',), ('Ym',)): 283, (('S1',), ('Zp',)): 103, (('S1',), ('Zm',)): 127, (('S1',), ('Xp',)): 176, (('S1',), ('Xm',)): 470, (('S1',), ('Yp',)): 108, (('S1',), ('Ym',)): 573, (('S2',), ('Zp',)): 99, (('S2',), ('Zm',)): 153, (('S2',), ('Xp',)): 221, (('S2',), ('X

{'01': {(('S0',), ('Zm',)): 1, (('S0',), ('Zp',)): 2, (('S0',), ('Xm',)): 1, (('S0',), ('Xp',)): 1, (('S0',), ('Yp',)): 4, (('S1',), ('Zp',)): 3242, (('S1',), ('Xp',)): 1670, (('S1',), ('Xm',)): 1665, (('S1',), ('Yp',)): 1559, (('S1',), ('Ym',)): 1659, (('S2',), ('Zp',)): 3356, (('S2',), ('Zm',)): 5, (('S2',), ('Xp',)): 1594, (('S2',), ('Xm',)): 1752, (('S2',), ('Ym',)): 1681, (('S2',), ('Yp',)): 1744, (('S3',), ('Zp',)): 3255, (('S3',), ('Xp',)): 1601, (('S3',), ('Xm',)): 1640, (('S3',), ('Yp',)): 1536, (('S3',), ('Ym',)): 1620, (('S0',), ('Ym',)): 0, (('S1',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '10': {(('S0',), ('Zm',)): 2, (('S0',), ('Zp',)): 5010, (('S0',), ('Xp',)): 2401, (('S0',), ('Xm',)): 2520, (('S0',), ('Ym',)): 2421, (('S0',), ('Yp',)): 2432, (('S1',), ('Zm',)): 2, (('S1',), ('Zp',)): 1607, (('S1',), ('Xp',)): 828, (('S1',), ('Xm',)): 894, (('S1',), ('Yp',)): 832, (('S1',), ('Ym',)): 823, (('S2',), ('Zm',)): 1, (('S2',), ('Zp',)): 1505, (('S2',), ('Xm',)): 797, (('S2',), (

...

{'00': {(('S0',), ('Zm',)): 2, (('S0',), ('Zp',)): 9993, (('S0',), ('Xp',)): 4850, (('S0',), ('Xm',)): 5143, (('S0',), ('Ym',)): 5010, (('S0',), ('Yp',)): 4983, (('S1',), ('Zp',)): 3315, (('S1',), ('Xm',)): 1704, (('S1',), ('Xp',)): 1650, (('S1',), ('Ym',)): 1758, (('S1',), ('Yp',)): 1713, (('S2',), ('Zp',)): 3244, (('S2',), ('Xm',)): 1618, (('S2',), ('Xp',)): 1545, (('S2',), ('Ym',)): 1613, (('S2',), ('Yp',)): 1571, (('S3',), ('Zp',)): 3358, (('S3',), ('Xm',)): 1701, (('S3',), ('Xp',)): 1694, (('S3',), ('Yp',)): 1702, (('S3',), ('Ym',)): 1671, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '10': {(('S0',), ('Zp',)): 2, (('S0',), ('Xp',)): 1, (('S0',), ('Xm',)): 1, (('S0',), ('Yp',)): 1, (('S1',), ('Zm',)): 5, (('S1',), ('Xp',)): 2, (('S1',), ('Ym',)): 1, (('S2',), ('Zp',)): 1, (('S2',), ('Zm',)): 2, (('S2',), ('Xm',)): 2, (('S2',), ('Yp',)): 1, (('S3',), ('Zm',)): 1, (('S3',), ('Zp',)): 1, (('S3',), ('Xp',)): 1, (('S3',), ('Yp',)): 3, (('S0',), ('Zm',)): 0, (('S

{'00000': {(('S0',), ('Zp',)): 2, (('S0',), ('Xp',)): 7, (('S0',), ('Yp',)): 5, (('S1',), ('Zm',)): 3, (('S1',), ('Xp',)): 3, (('S1',), ('Xm',)): 3, (('S2',), ('Zp',)): 1, (('S2',), ('Zm',)): 2, (('S2',), ('Xp',)): 2, (('S2',), ('Xm',)): 2, (('S2',), ('Yp',)): 3, (('S2',), ('Ym',)): 2, (('S3',), ('Zp',)): 3, (('S3',), ('Zm',)): 3, (('S3',), ('Xp',)): 2, (('S3',), ('Xm',)): 2, (('S3',), ('Yp',)): 2, (('S3',), ('Ym',)): 1, (('S0',), ('Zm',)): 0, (('S0',), ('Xm',)): 0, (('S0',), ('Ym',)): 0, (('S1',), ('Zp',)): 0, (('S1',), ('Yp',)): 0, (('S1',), ('Ym',)): 0}, '00001': {(('S0',), ('Zp',)): 145, (('S0',), ('Zm',)): 4, (('S0',), ('Xp',)): 116, (('S0',), ('Xm',)): 113, (('S0',), ('Yp',)): 114, (('S0',), ('Ym',)): 97, (('S1',), ('Zp',)): 85, (('S1',), ('Zm',)): 124, (('S1',), ('Xp',)): 45, (('S1',), ('Xm',)): 141, (('S1',), ('Yp',)): 32, (('S1',), ('Ym',)): 169, (('S2',), ('Zp',)): 84, (('S2',), ('Zm',)): 139, (('S2',), ('Xp',)): 60, (('S2',), ('Xm',)): 141, (('S2',), ('Yp',)): 176, (('S2',),

{'00000': {(('S0',), ('Zp',)): 640, (('S0',), ('Zm',)): 20, (('S0',), ('Xp',)): 271, (('S0',), ('Xm',)): 298, (('S0',), ('Yp',)): 283, (('S0',), ('Ym',)): 289, (('S1',), ('Zp',)): 185, (('S1',), ('Zm',)): 379, (('S1',), ('Xp',)): 148, (('S1',), ('Xm',)): 402, (('S1',), ('Yp',)): 88, (('S1',), ('Ym',)): 487, (('S2',), ('Zp',)): 195, (('S2',), ('Zm',)): 345, (('S2',), ('Xp',)): 201, (('S2',), ('Xm',)): 374, (('S2',), ('Yp',)): 486, (('S2',), ('Ym',)): 62, (('S3',), ('Zp',)): 173, (('S3',), ('Zm',)): 418, (('S3',), ('Xp',)): 478, (('S3',), ('Xm',)): 42, (('S3',), ('Yp',)): 215, (('S3',), ('Ym',)): 339}, '00001': {(('S0',), ('Zp',)): 530, (('S0',), ('Zm',)): 14, (('S0',), ('Xp',)): 283, (('S0',), ('Xm',)): 273, (('S0',), ('Yp',)): 302, (('S0',), ('Ym',)): 281, (('S1',), ('Zp',)): 205, (('S1',), ('Zm',)): 338, (('S1',), ('Xp',)): 163, (('S1',), ('Xm',)): 428, (('S1',), ('Yp',)): 84, (('S1',), ('Ym',)): 440, (('S2',), ('Zp',)): 223, (('S2',), ('Zm',)): 346, (('S2',), ('Xp',)): 151, (('S2',),

{'11': {(('S0',), ('Zp',)): 1, (('S0',), ('Xm',)): 1, (('S0',), ('Ym',)): 1, (('S0',), ('Yp',)): 2, (('S1',), ('Zp',)): 6607, (('S1',), ('Xp',)): 3235, (('S1',), ('Xm',)): 3337, (('S1',), ('Ym',)): 3389, (('S1',), ('Yp',)): 3281, (('S2',), ('Zp',)): 6808, (('S2',), ('Zm',)): 2, (('S2',), ('Xp',)): 3319, (('S2',), ('Xm',)): 3434, (('S2',), ('Yp',)): 3378, (('S2',), ('Ym',)): 3391, (('S3',), ('Zp',)): 6641, (('S3',), ('Xp',)): 3230, (('S3',), ('Xm',)): 3393, (('S3',), ('Yp',)): 3284, (('S3',), ('Ym',)): 3334, (('S0',), ('Zm',)): 0, (('S0',), ('Xp',)): 0, (('S1',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '01': {(('S0',), ('Zm',)): 2, (('S0',), ('Xm',)): 1, (('S0',), ('Xp',)): 2, (('S1',), ('Zp',)): 1, (('S1',), ('Zm',)): 1, (('S1',), ('Xm',)): 4, (('S1',), ('Ym',)): 2, (('S2',), ('Zp',)): 5, (('S2',), ('Xp',)): 2, (('S2',), ('Yp',)): 1, (('S2',), ('Ym',)): 1, (('S3',), ('Xm',)): 1, (('S3',), ('Yp',)): 1, (('S0',), ('Zp',)): 0, (('S0',), ('Yp',)): 0, (('S0',), ('Ym',)): 0, (('S1',), ('Xp',)):

{'00000': {(('S0',), ('Zp',)): 7, (('S0',), ('Xp',)): 7, (('S0',), ('Xm',)): 5, (('S0',), ('Yp',)): 1, (('S0',), ('Ym',)): 8, (('S1',), ('Zp',)): 5, (('S1',), ('Zm',)): 7, (('S1',), ('Xp',)): 5, (('S1',), ('Xm',)): 2, (('S1',), ('Yp',)): 5, (('S1',), ('Ym',)): 5, (('S2',), ('Zp',)): 3, (('S2',), ('Zm',)): 4, (('S2',), ('Xp',)): 3, (('S2',), ('Xm',)): 7, (('S2',), ('Yp',)): 5, (('S2',), ('Ym',)): 4, (('S3',), ('Zp',)): 1, (('S3',), ('Zm',)): 3, (('S3',), ('Xm',)): 1, (('S3',), ('Yp',)): 3, (('S3',), ('Ym',)): 4, (('S0',), ('Zm',)): 0, (('S3',), ('Xp',)): 0}, '00001': {(('S0',), ('Zp',)): 667, (('S0',), ('Zm',)): 20, (('S0',), ('Xp',)): 147, (('S0',), ('Xm',)): 154, (('S0',), ('Yp',)): 158, (('S0',), ('Ym',)): 154, (('S1',), ('Zp',)): 121, (('S1',), ('Zm',)): 230, (('S1',), ('Xp',)): 101, (('S1',), ('Xm',)): 245, (('S1',), ('Yp',)): 39, (('S1',), ('Ym',)): 269, (('S2',), ('Zp',)): 110, (('S2',), ('Zm',)): 234, (('S2',), ('Xp',)): 78, (('S2',), ('Xm',)): 262, (('S2',), ('Yp',)): 282, (('S

In [66]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [68]:
from utils.utils import filter_results

In [69]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [70]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [71]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [72]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [73]:
final_expect_val

(-3.3413295609689913+0j)

In [74]:
import json
with open('BeH2_auckland_check4.json', 'w') as f:
    json.dump(norm_filtered_lists, f)